In [1]:
import os, sys

# Get the absolute path of the notebook's directory
notebook_dir = os.getcwd()

# Navigate to the parent directory
parent_dir = os.path.abspath(os.path.join(notebook_dir, "../.."))

# Add the parent directory to sys.path so we can import modules
sys.path.append(parent_dir)

# Verify the path
print(f"Added to sys.path: {parent_dir}")

# Add it to sys.path
sys.path.append(parent_dir)

print(parent_dir)
from localityaware.module import *
from NSGA_nn.nsga import *
from FlashOperation.Refrig2DrumHeatExConstrDummy import Refrig2DrumConstraintHeatExConstDummy

Added to sys.path: /Users/panwapromtep/Library/CloudStorage/OneDrive-JohnsHopkins/Process Design/Aspen
/Users/panwapromtep/Library/CloudStorage/OneDrive-JohnsHopkins/Process Design/Aspen
['/Users/panwapromtep/miniforge3/envs/torch/lib/python312.zip', '/Users/panwapromtep/miniforge3/envs/torch/lib/python3.12', '/Users/panwapromtep/miniforge3/envs/torch/lib/python3.12/lib-dynload', '', '/Users/panwapromtep/miniforge3/envs/torch/lib/python3.12/site-packages', '/Users/panwapromtep/Library/CloudStorage/OneDrive-JohnsHopkins/Process Design/Aspen', '/Users/panwapromtep/Library/CloudStorage/OneDrive-JohnsHopkins/Process Design/Aspen', '/Users/panwapromtep/Library/CloudStorage/OneDrive-JohnsHopkins/Process Design/Aspen']


In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import numpy as np
import pickle
import os
import time

# Define the file path for saving/loading the data
data_file = "flash_simulation_data_sim_new.pkl"

# Define parameter ranges
flash_1_range_sim = np.linspace(-15, 15, 10)
flash_2_range_sim = np.linspace(-15, 15, 10)

print("Generating new data...")
assSim = Refrig2DrumConstraintHeatExConstDummy(AspenFile="../FlashOperation/FlashOperation_HeatExchanger.bkp",
                                        wdpath="../FlashOperation",
                                        visibility=False,
                                        Penalty=1e3)

start_time = time.time()  # Start timing
data_sim = []
for flash_1 in flash_1_range_sim:
    for flash_2 in flash_2_range_sim:
        x_unflat = assSim.unflatten_params([float(flash_1), float(flash_2)])
        data_sim.append([flash_1, flash_2, assSim.run_obj(x_unflat)])

data_sim = np.array(data_sim)
data_gen_time = time.time() - start_time  # Total time to generate data
total_original_assSim_calls = len(data_sim)

Generating new data...
⚠️  Dummy Mode: Rosenbrock function in place of AspenSim.


In [4]:
miny = np.min(data_sim[:, 2])
maxy = np.max(data_sim[:, 2])
print("min max y:", miny, maxy)

min max y: 123.90123456790182 5760256.0


In [5]:

scaler = TorchMinMaxScaler((-1, 1), 
                           max_vals=[15.0, 15.0],
                           min_vals=[-15.0, -15.0],
                           min_y=0,
                           max_y=1e7 ,
                           scale_y=True)
#scale data_sim 
data_sim_xscaled, data_sim_yscaled = scaler.transform(data_sim[:, :2], data_sim[:, 2])

#recombine the data
data_sim_scaled = np.column_stack([data_sim_xscaled, data_sim_yscaled])

# **Initialize Model & Datasets**
dataset = DynamicDataset(data_sim_scaled)
model = MLP(2, [20,20, 20, 20], 1)

In [6]:
problem = FlashOpProblemNN(model=model)

In [7]:
out = optimize_surr_ga(model=model,
                   dataset=dataset,
                   assSim=assSim,
                   problem=problem,
                   lrs={'first':1e-4, 'others':1e-4},
                   epochs={'first':3000, 'others':600},
                   min_vals=scaler.min_x,
                   max_vals=scaler.max_x,
                   scaler=scaler,
                   device='cpu',
                   iter=6,
                   print_loss=False,
                   print_it_data=True,
                   pop_size=1000,
                   n_gen = 20,
                   new_data_size=20,
                   batch_size=128
                   )

Iteration 0: Training surrogate model...
n_gen  |  n_eval  |     f_avg     |     f_min    
     1 |     1000 | -7.928882E-01 | -1.039387E+00
     2 |     2000 | -1.008169E+00 | -1.039387E+00
     3 |     3000 | -1.023396E+00 | -1.039387E+00
     4 |     4000 | -1.030457E+00 | -1.039387E+00
     5 |     5000 | -1.034810E+00 | -1.039535E+00
     6 |     6000 | -1.037462E+00 | -1.039539E+00
     7 |     7000 | -1.038626E+00 | -1.039539E+00
     8 |     8000 | -1.039069E+00 | -1.039540E+00
     9 |     9000 | -1.039267E+00 | -1.039541E+00
    10 |    10000 | -1.039383E+00 | -1.039541E+00
    11 |    11000 | -1.039448E+00 | -1.039541E+00
    12 |    12000 | -1.039485E+00 | -1.039541E+00
    13 |    13000 | -1.039506E+00 | -1.039541E+00
    14 |    14000 | -1.039520E+00 | -1.039541E+00
    15 |    15000 | -1.039528E+00 | -1.039541E+00
    16 |    16000 | -1.039532E+00 | -1.039541E+00
    17 |    17000 | -1.039535E+00 | -1.039541E+00
    18 |    18000 | -1.039537E+00 | -1.039541E+00
    19 | 